In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn import metrics

In [2]:
def process_dfs(train_file):
    train_df=pd.read_csv(train_file)
    train_df=train_df.drop('FL_DATE', axis=1)
    train_df=train_df.drop('ORIGIN_CITY_NAME', axis=1)
    train_df=train_df.drop('DEST_CITY_NAME', axis=1)
    train_df=train_df.drop('Unnamed: 13', axis=1)
    train_df=train_df.drop('ORIGIN', axis=1)
    train_df=train_df.drop('UNIQUE_CARRIER', axis=1)
    train_df=train_df.drop('DEST', axis=1)
    train_df=one_hot_encodings(train_df)
    
    return train_df

def one_hot_encodings(df):
    for item in df.columns:
        if df[item].dtypes=='O':
            df[item]=df[item].astype('category')
    df=pd.get_dummies(df)
    return df

In [3]:
train= process_dfs("./aggregated.csv")

In [4]:
train.head()

,MONTH,DAY_OF_WEEK,FL_NUM,CRS_DEP_TIME,ARR_DEL15,CRS_ELAPSED_TIME,DISTANCE
0,2.0,6.0,28.0,1000.0,0.0,156.0,937.0
1,2.0,7.0,28.0,739.0,0.0,153.0,937.0
2,2.0,1.0,28.0,1028.0,0.0,158.0,937.0
3,2.0,2.0,28.0,739.0,0.0,153.0,937.0
4,2.0,3.0,33.0,1907.0,0.0,90.0,266.0


In [5]:
training=train.iloc[1:4103483, :]
validation=train.iloc[4103483:5129354, :]

In [6]:
np.any(np.isnan(training))

True

In [7]:
#training=training.fillna(training.mean())
#validation=validation.fillna(validation.mean())
training=pd.DataFrame(training).fillna(0)
validation=pd.DataFrame(validation).fillna(0)

In [8]:
training.to_csv('./training.csv')
validation.to_csv('./validation.csv')

In [9]:
training=pd.read_csv('./training.csv')
validation=pd.read_csv('./validation.csv')

In [ ]:
validation.head()

,Unnamed: 0,MONTH,DAY_OF_WEEK,FL_NUM,CRS_DEP_TIME,ARR_DEL15,CRS_ELAPSED_TIME,DISTANCE
0,4103483,9.0,5.0,2923.0,800.0,0.0,105.0,623.0
1,4103484,9.0,5.0,4251.0,1855.0,0.0,105.0,623.0
2,4103485,9.0,5.0,791.0,1335.0,0.0,95.0,562.0
3,4103486,9.0,5.0,1747.0,2115.0,0.0,95.0,562.0
4,4103487,9.0,5.0,3199.0,1005.0,0.0,95.0,562.0


In [ ]:
rf2=RandomForestClassifier(n_estimators=50, min_samples_split=50)
from sklearn.model_selection import GridSearchCV

params = {
    'max_features': [0.25, 0.5],
    'max_depth' : [5, 10]
}
gs = GridSearchCV(cv=5, param_grid=params, estimator=rf2, verbose=3)
gs.fit(training.drop('ARR_DEL15', axis=1), training['ARR_DEL15'])
print(gs.best_params_, gs.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] max_depth=5, max_features=0.25 ..................................
[CV]  max_depth=5, max_features=0.25, score=0.820690218192585, total= 1.7min
[CV] max_depth=5, max_features=0.25 ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.8min remaining:    0.0s


[CV]  max_depth=5, max_features=0.25, score=0.482213289435687, total= 1.6min
[CV] max_depth=5, max_features=0.25 ..................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.6min remaining:    0.0s


[CV]  max_depth=5, max_features=0.25, score=0.8206899997075653, total= 1.6min
[CV] max_depth=5, max_features=0.25 ..................................
[CV]  max_depth=5, max_features=0.25, score=0.8206899997075653, total= 1.7min
[CV] max_depth=5, max_features=0.25 ..................................
[CV]  max_depth=5, max_features=0.25, score=0.8206899997075653, total= 1.5min
[CV] max_depth=5, max_features=0.5 ...................................
[CV]  max_depth=5, max_features=0.5, score=0.820690218192585, total= 3.2min
[CV] max_depth=5, max_features=0.5 ...................................
[CV]  max_depth=5, max_features=0.5, score=0.482213289435687, total= 3.1min
[CV] max_depth=5, max_features=0.5 ...................................
[CV]  max_depth=5, max_features=0.5, score=0.8206899997075653, total= 3.0min
[CV] max_depth=5, max_features=0.5 ...................................
[CV]  max_depth=5, max_features=0.5, score=0.3985409944729839, total= 3.4min
[CV] max_depth=5, max_features=0.5

In [ ]:
pd.DataFrame(gs.grid_scores_).sort_values(by=['mean_validation_score'], ascending=False)